In [57]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
import torch
import torch.nn as nn
import torch.nn.functional as F #functional has the activation functions


In [58]:
#What is A nerual Network? What are the general 
#steps required to build a neural network?

# A process that mimics the human brain to understand patterns
#in data. The discipline takes programming concepts & applies them
#to linguistics. Some general steps required to build a neural network
#is choosing a loss function and training the model.

In [59]:
# Generally, how do you check the performance of a neural network? Why is this the case?

# You can check the performance by using various methods
# precision and recall, loss functions, F1 score, confusion
# matrix, ect. These methods can look at different aspects
# of neural networks.

In [60]:
def get_df():
   

    
    credit_df = pd.read_csv("../week_10/Credit_card.csv", encoding_errors='ignore', delimiter = ',', on_bad_lines='skip' )
    credit_label_df = pd.read_csv("../week_10/Credit_card_label.csv", encoding_errors='ignore', delimiter = ',', on_bad_lines='skip' )
    
    merged_df = credit_df.merge(credit_label_df, on='Ind_ID')
    new_merged_df = merged_df.dropna()
    #Something different that I did compared to the last time
    # Is dropping some columns from the dataframe.
    new_merged_df.drop(['Ind_ID','Annual_income','Birthday_count','Employed_days'], axis = 'columns', inplace=True)
    return new_merged_df

credit_df = get_df()

C:\Users\toriw\AppData\Local\Temp\ipykernel_27564\2040202022.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_merged_df.drop(['Ind_ID','Annual_income','Birthday_count','Employed_days'], axis = 'columns', inplace=True)


In [61]:
credit_df

,GENDER,Car_Owner,Propert_Owner,CHILDREN,Type_Income,EDUCATION,Marital_status,Housing_type,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
8,M,Y,Y,1,Commercial associate,Secondary / secondary special,Married,House / apartment,1,0,1,1,Core staff,3,1
9,M,Y,Y,1,Pensioner,Secondary / secondary special,Married,House / apartment,1,0,1,1,Core staff,3,1
10,M,Y,Y,1,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,0,1,1,Core staff,1,1
11,F,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,1,1,1,0,Cooking staff,2,1
13,F,Y,Y,0,Working,Secondary / secondary special,Married,House / apartment,1,1,1,0,Cooking staff,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,F,Y,Y,1,Working,Higher education,Married,House / apartment,1,0,0,0,Accountants,3,0
1542,M,Y,N,1,State servant,Secondary / secondary special,Married,House / apartment,1,0,1,0,Drivers,3,0
1544,F,N,N,0,Commercial associate,Incomplete higher,Single / not married,House / apartment,1,0,0,0,Accountants,1,0
1545,M,Y,Y,2,Working,Higher education,Married,House / apartment,1,0,0,0,Managers,4,0


In [62]:
def cleaning_data(credit_df):
    credit_df['GENDER'] = np.where(credit_df['GENDER'] == 'F', 1, 0)
    credit_df['Car_Owner'] = np.where(credit_df['Car_Owner'] == 'M', 1, 0)
    credit_df['Propert_Owner'] = np.where(credit_df['Propert_Owner'] == 'N', 1, 0)
   
    class_labels = LabelEncoder()
    credit_df['Type_Income'] = class_labels.fit_transform(credit_df['Type_Income'].values)
    credit_df['EDUCATION'] = class_labels.fit_transform(credit_df['EDUCATION'].values)
    credit_df['Marital_status'] = class_labels.fit_transform(credit_df['Marital_status'].values)

    #housing_map = {'House/ apartment':1,'With parents':2,'Municipal apartment':3,'Rented apartment':4,'Office apartment':5}
    #credit_df['Housing_type'] = credit_df['Housing_type'].map(housing_map)
    credit_df['Housing_type'] = np.where(credit_df['Housing_type'] == 'Housing / apartment', 1, 0)

    #credit_df['Housing_type'] = class_labels.fit_transform(credit_df['Housing_type'].values)
    credit_df['Type_Occupation'] = class_labels.fit_transform(credit_df['Type_Occupation'].values)
    return credit_df

In [63]:
cleaning_data(credit_df)

,GENDER,Car_Owner,Propert_Owner,CHILDREN,Type_Income,EDUCATION,Marital_status,Housing_type,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
8,0,0,0,1,0,3,1,0,1,0,1,1,3,3,1
9,0,0,0,1,1,3,1,0,1,0,1,1,3,3,1
10,0,0,0,1,0,3,3,0,1,0,1,1,3,1,1
11,1,0,0,0,3,3,1,0,1,1,1,0,2,2,1
13,1,0,0,0,3,3,1,0,1,1,1,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1540,1,0,0,1,3,0,1,0,1,0,0,0,0,3,0
1542,0,0,1,1,2,3,1,0,1,0,1,0,4,3,0
1544,1,0,1,0,0,1,3,0,1,0,0,0,0,1,0
1545,0,0,0,2,3,0,1,0,1,0,0,0,10,4,0


In [64]:

X = credit_df.drop('label', axis=1).values
y = credit_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=32, stratify=y)

   #create tensors from our data
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
print(X_train)

tensor([[ 0.,  0.,  0.,  ...,  0.,  8.,  2.],
        [ 0.,  0.,  1.,  ...,  0., 10.,  2.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  2.],
        ...,
        [ 1.,  0.,  0.,  ...,  1.,  2.,  3.],
        [ 1.,  0.,  0.,  ...,  0., 14.,  2.],
        [ 0.,  0.,  0.,  ...,  0.,  4.,  2.]])


In [65]:
#artificial neural network
class ANN_Model(nn.Module):
    def __init__(self, input_features=8,hidden1=20,hidden2=20,out_features=2):
        super().__init__() #super is a computed indirected reference
        # what that means is that it isolates changes and makes 
        # sure the children in the layers of multiple inheritnce are calling
        # the right parents
        self.layer_1_connection = nn.Linear(input_features,hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)

    def forward(self, x):
        #apply activation function
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [66]:
torch.manual_seed(32)

#create an instance of the model
ann = ANN_Model()

In [67]:
# loss function
loss_function = nn.CrossEntropyLoss()


#optimizer
optimizer = torch.optim.Adam(ann.parameters(), lr=0.01)

In [68]:
#run model through various epochs/iterations
final_loss=[]
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = ann.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)

    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss}')

    optimizer.zero_grad() #zeros the gradient before running backward propagation
    loss.backward() #minimize the loss function using backward propagation
    optimizer.step() #perform one optimization step each function

RuntimeError: mat1 and mat2 shapes cannot be multiplied (820x14 and 8x20)

In [69]:
# predictions
y_pred = []

with torch.no_grad(): #decreases memory consumption
    for i, data in enumerate(X_test):
        prediction = ann(data)
        y_pred.append(prediction.argmax()) #returns index with the max
        # element in each position set

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x14 and 8x20)

In [ ]:
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)